In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.io as pio
pio.renderers.default='notebook'
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as matplot

import re
import sklearn
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

**Merging data**

In [ ]:
#Merging 2013-14 datsets
df1 = pd.read_csv('../input/national-health-and-nutrition-examination-survey/labs.csv')
df2 = pd.read_csv('../input/national-health-and-nutrition-examination-survey/examination.csv')
df3 = pd.read_csv('../input/national-health-and-nutrition-examination-survey/demographic.csv')
df4a = pd.read_csv('../input/national-health-and-nutrition-examination-survey/diet.csv')
df5 = pd.read_csv('../input/national-health-and-nutrition-examination-survey/questionnaire.csv')

df2.drop(['SEQN'], axis = 1, inplace=True)
df3.drop(['SEQN'], axis = 1, inplace=True)
df4a.drop(['SEQN'], axis = 1, inplace=True)
df5.drop(['SEQN'], axis = 1, inplace=True)

dfa = pd.concat([df1, df2], axis=1, join='inner')
dfa = pd.concat([dfa, df3], axis=1, join='inner')
dfa = pd.concat([dfa, df4a], axis=1, join='inner')
dfa = pd.concat([dfa, df5], axis=1, join='inner')


In [ ]:
dfa


In [ ]:
#reading 2015-16 healthdata
d56 = pd.read_csv('../input/merged/merged_2015_2016.csv')
d56.describe()

In [ ]:
#reading 2017-18 healthdata
d78 = pd.read_csv('../input/merged/merged_2017_2018.csv')
d78.describe()


In [ ]:
a=d56.columns

b=d78.columns
a

In [ ]:
#merging 2015,2016,2017,2018 data
df_m = pd.concat([d56,d78],ignore_index=True,axis=0)

df_m.describe()
df_m

df_m = pd.concat([d56,d78[a]],ignore_index=True,axis=0)
'DR1TWS', 'DMDHRMAR', 'DMDHRBR4', 'DMDHREDU', 'DMDHRAGE', 'DMDHSEDU'
df_m = pd.concat([d78,d56[b]],ignore_index=True,axis=0)
'DMDHRAGZ', 'DMDHREDZ', 'DMDHRMAZ', 'DMDHSEDZ', 'DR1TWSZ'

**merged 2015,16,17,18**

In [ ]:
#Dropiing non common rows across different datasets
#
df_md=df_m.drop(['DR1TWS', 'DMDHRMAR', 'DMDHRBR4', 'DMDHREDU', 'DMDHRAGE', 'DMDHSEDU','DMDHRAGZ', 'DMDHREDZ', 'DMDHRMAZ', 'DMDHSEDZ', 'DR1TWSZ','DR1_330Z', 'DR1_320Z', 'DR1MRESP', 'DR1_300', 'DR1HELP'],axis=1)
df_md


In [ ]:
df_mdc=df_md.columns
df_mdc

In [ ]:
#dfa-2013-14dataframe
print(dfa[df_mdc])

**Merged data frame**

In [ ]:
#Merging 2013,14,15,16,17,18 data
df_x = pd.concat([dfa[df_mdc],df_md],ignore_index=True,axis=0)
df_x.dropna(axis=1, how='all')
df_x.dropna(axis=0, how='all')
df_x

In [ ]:
#Renaming required columns
df_x = df_x.rename(columns = {'SEQN' : 'ID',
                          'RIDAGEYR': 'Age',
                          'RIAGENDR' : 'Gender',
                          'WTINT2YR' : 'Weight',
                          'INDFMIN2' : 'Tot_family_income',
                          'DMDFMSIZ'  :'Tot_no_fam_members',
                          'DMDYRSUS' : 'Years_in_US', # Nan -> american i guess
                          'INDFMPIR' : 'Family_income',
                          'LBXGH' : 'GlycoHemoglobin',
                          'BMXARMC' : 'ArmCircum',
                          'BMDAVSAD' : 'SaggitalAbdominal',
                          'MGDCGSZ' : 'GripStrength',
                          'DRABF' : 'Breast_fed',
                           'MCQ053':'Anemia_treatment',
                            'LBXHGB': 'Hemoglobin',
                            'MCQ220': 'Cancer',
                            'BMXBMI': 'BMI',
                            'DMDEDUC2':'Educationlevel',
                            'INDHHINC': 'annualincome',
                            'RIDRETH1':'Ethinicity',
                            
                           })

df_x.describe()

In [ ]:
#Filling continous and categorical missing values
df_x['Tot_family_income'] = df_x['Tot_family_income'].interpolate(limit_direction ='both')
df_x['Hemoglobin'] = df_x['Hemoglobin'].interpolate(limit_direction ='both')
df_x['Age'] = df_x['Age'].interpolate(limit_direction ='both')
df_x['Weight'] = df_x['Weight'].interpolate(limit_direction ='both')
df_x['Tot_no_fam_members'] = df_x['Tot_no_fam_members'].interpolate(limit_direction ='both')
    
df_x = df_x.fillna(df_x['Gender'].value_counts().index[0])
df_x = df_x.fillna(df_x['Anemia_treatment'].value_counts().index[0])
df_x.describe()
    

    

In [ ]:
#Filling rest of missing values
import pandas as pd
import numpy as np
from sklearn.base import TransformerMixin
class DataFrameImputer(TransformerMixin):
    def __init__(self):
        """Impute missing values.
        Columns of dtype object are imputed with the most frequent value
        in column.
        Columns of other types are imputed with mean of column.
        """
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)


X = df_x
print('before...')
print(X)
df_x = DataFrameImputer().fit_transform(X)
print('after...')
print(df_x)

**Anemia calculation**

In [ ]:
df_x.loc[(df_x['Gender']==1) & (df_x['Hemoglobin'] < 13), 'Anemia'] = 1
df_x.loc[(df_x['Gender']==1) & (df_x['Hemoglobin'] > 13), 'Anemia'] = 0

df_x.loc[(df_x['Gender']==2) & (df_x['Hemoglobin'] < 12), 'Anemia'] = 1
df_x.loc[(df_x['Gender']==2) & (df_x['Hemoglobin'] > 12), 'Anemia'] = 0

print(df_x)

In [ ]:
df_x['Anemia']=df_x['Anemia'].fillna(0)
print(df_x)

In [ ]:
#Distribution of ANEMIA values
df_x['Anemia'].value_counts(normalize=True)

**Percentage of null values**

In [ ]:
null=100*(df_x.isnull().sum())/(df_x.shape[0])

df4ad_null=pd.DataFrame({'percentage':null})

df4ad_null.describe()

**Extracting important values from dataframe**

In [ ]:
df_m1 = df_x.loc[:,['DR1DRSTZ',
 'DRD350A',
 'DRD350C',
 'DRD350D',
 'DRD350E',
 'DRD350G',
 'DRD350I',
 'DRD350J',
 'DRD350K',
 'DRD360',
 'DRD370C',
 'DRD370G',
 'DRD370H',
 'DRD370I',
 'DRD370J',
 'DRD370K',
 'DRD370L',
 'DRD370N',
 'DRD370O',
 'DRD370P',
 'DRD370Q',
 'DRD370R',
 'DRD370S',
 'DRD370T',
 'DRD370U',
 'DRD370V',
 'ID',
 'Gender',
 'Age',
 'Weight',
 'Tot_family_income',
 'Tot_no_fam_members',
 'Hemoglobin',
 'Anemia_treatment',
 'Cancer',
 'BMI',
 'Educationlevel',
 'Ethinicity',
 'Breast_fed',
 'GlycoHemoglobin',
 'Anemia'
    ] ]
df_m1.describe()
print(df_m1)
#removed drsthepd,drespnd,drabf

In [ ]:
nullv=100*(df_m1.isnull().sum())/(df_m1.shape[0])

var_allnull=pd.DataFrame({'percentage':nullv})
print(var_allnull)

In [ ]:
#Dropping ID column
df_m1.drop('ID',
  axis='columns', inplace=True)


In [ ]:




#deopping hemoglobin
X = df_m1.drop(['Anemia','Hemoglobin'],axis=1)
y = df_m1[['Anemia']]

In [ ]:
#Split the dataset into train and Test
from sklearn.model_selection import train_test_split

seed = 195
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

**RFE using Random Forest regressor-Top 30 features**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE

# Init the transformer
rfe = RFE(estimator=RandomForestRegressor(), n_features_to_select=30)

# Fit to the training data
_ = rfe.fit(X, y)
newvar=X.loc[:, rfe.support_]

newvar.describe()

In [ ]:
#Adding anemia to top 30 variables
newvar["Anemia"] = df_m1[["Anemia"]]
print(newvar)

In [ ]:
newvar.describe()

**Heat map-Top 30 features**

In [ ]:
colormap = plt.cm.viridis
plt.figure(figsize=(20,20))
sns.heatmap(newvar.astype(float).corr(), linewidths=0.1, vmax=1.0, square=True, cmap=colormap, annot=True)

In [ ]:
#deopping hemoglobin
X = newvar.drop(['Anemia','Weight','Anemia_treatment'],axis=1)
y = newvar[['Anemia']]

In [ ]:
X

In [ ]:
#Split the dataset into train and Test
from sklearn.model_selection import train_test_split

seed = 195
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [ ]:
#scaled_pos_weight-calculation
ratio= float(np.sum(y_train == 0)) / np.sum(y_train == 1)
ratio


**XGBClassifier-Optuna**

In [ ]:
import xgboost as xgb
from optuna import create_study, logging
from optuna.pruners import MedianPruner
from optuna.integration import XGBoostPruningCallback


In [ ]:
import timeit
import pickle
import sys
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, \
                            precision_recall_curve, roc_curve, accuracy_score
from sklearn.exceptions import NotFittedError

**UTILITY FUNCTIONS**

In [ ]:
def confusion_plot(matrix, labels=None):
    """ Display binary confusion matrix as a Seaborn heatmap """
    
    labels = labels if labels else ['Negative (0)', 'Positive (1)']
    
    fig, ax = plt.subplots(nrows=1, ncols=1)
    sns.heatmap(data=matrix, cmap='Blues', annot=True, fmt='d',
                xticklabels=labels, yticklabels=labels, ax=ax)
    ax.set_xlabel('PREDICTED')
    ax.set_ylabel('ACTUAL')
    ax.set_title('Confusion Matrix')
    plt.close()
    
    return fig

In [ ]:
def roc_plot(y_true, y_probs, label, compare=False, ax=None):
    """ Plot Receiver Operating Characteristic (ROC) curve 
        Set `compare=True` to use this function to compare classifiers. """
    
    fpr, tpr, thresh = roc_curve(y_true, y_probs,
                                 drop_intermediate=False)
    auc = round(roc_auc_score(y_true, y_probs), 2)
    
    fig, axis = (None, ax) if ax else plt.subplots(nrows=1, ncols=1)
    label = ' '.join([label, f'({auc})']) if compare else None
    sns.lineplot(x=fpr, y=tpr, ax=axis, label=label)
    
    if compare:
        axis.legend(title='Classifier (AUC)', loc='lower right')
    else:
        axis.text(0.72, 0.05, f'AUC = { auc }', fontsize=12,
                  bbox=dict(facecolor='green', alpha=0.4, pad=5))
            
        # Plot No-Info classifier
        axis.fill_between(fpr, fpr, tpr, alpha=0.3, edgecolor='g',
                          linestyle='--', linewidth=2)
        
    axis.set_xlim(0, 1)
    axis.set_ylim(0, 1)
    axis.set_title('ROC Curve')
    axis.set_xlabel('False Positive Rate [FPR]\n(1 - Specificity)')
    axis.set_ylabel('True Positive Rate [TPR]\n(Sensitivity or Recall)')
    
    plt.close()
    
    return axis if ax else fig

In [ ]:
def precision_recall_plot(y_true, y_probs, label, compare=False, ax=None):
    """ Plot Precision-Recall curve.
        Set `compare=True` to use this function to compare classifiers. """
    
    p, r, thresh = precision_recall_curve(y_true, y_probs)
    p, r, thresh = list(p), list(r), list(thresh)
    p.pop()
    r.pop()
    
    fig, axis = (None, ax) if ax else plt.subplots(nrows=1, ncols=1)
    
    if compare:
        sns.lineplot(r, p, ax=axis, label=label)
        axis.set_xlabel('Recall')
        axis.set_ylabel('Precision')
        axis.legend(loc='lower left')
    else:
        sns.lineplot(thresh, p, label='Precision', ax=axis)
        axis.set_xlabel('Threshold')
        axis.set_ylabel('Precision')
        axis.legend(loc='lower left')

        axis_twin = axis.twinx()
        sns.lineplot(thresh, r, color='limegreen', label='Recall', ax=axis_twin)
        axis_twin.set_ylabel('Recall')
        axis_twin.set_ylim(0, 1)
        axis_twin.legend(bbox_to_anchor=(0.24, 0.18))
    
    axis.set_xlim(0, 1)
    axis.set_ylim(0, 1)
    axis.set_title('Precision Vs Recall')
    
    plt.close()
    return axis if ax else fig    

In [ ]:
def train_clf(clf, x_train, y_train, sample_weight=None, refit=False):
    train_time = 0
    
    try:
        if refit:
            raise NotFittedError
        y_pred_train = clf.predict(x_train)
    except NotFittedError:
        start = timeit.default_timer()
        
        if sample_weight is not None:
            clf.fit(x_train, y_train, sample_weight=sample_weight)
        else:
            clf.fit(x_train, y_train)
        
        end = timeit.default_timer()
        train_time = end - start
        
        y_pred_train = clf.predict(x_train)
    
    train_acc = accuracy_score(y_train, y_pred_train)
    return clf, y_pred_train, train_acc, train_time

In [ ]:
def feature_importance_plot(importances, feature_labels, ax=None):
    fig, axis = (None, ax) if ax else plt.subplots(nrows=1, ncols=1, figsize=(5, 10))
    sns.barplot(x=importances, y=feature_labels, ax=axis)
    axis.set_title('Feature Importance Measures')
    
    plt.close()
    
    return axis if ax else fig


In [ ]:
def model_memory_size(clf):
    return sys.getsizeof(pickle.dumps(clf))
def report(clf, x_train, y_train, x_test, y_test, display_scores=[],
           sample_weight=None, refit=False, importance_plot=False,
           confusion_labels=None, feature_labels=None, verbose=True):
    """ Trains the passed classifier if not already trained and reports
        various metrics of the trained classifier """
    
    dump = dict()
    
    ## Train if not already trained
    clf, train_predictions, \
    train_acc, train_time = train_clf(clf, x_train, y_train,
                                      sample_weight=sample_weight,
                                      refit=refit)
    ## Testing
    start = timeit.default_timer()
    test_predictions = clf.predict(x_test)
    end = timeit.default_timer()
    test_time = end - start
    
    test_acc = accuracy_score(y_test, test_predictions)
    y_probs = clf.predict_proba(x_test)[:, 1]
    
    roc_auc = roc_auc_score(y_test, y_probs)
    ## Additional scores
    scores_dict = dict()
    for func in display_scores:
        scores_dict[func.__name__] = [func(y_train, train_predictions),
                                      func(y_test, test_predictions)]
        
    ## Model Memory
    model_mem = round(model_memory_size(clf) / 1024, 2)
    
    print(clf)
    print("\n=============================> TRAIN-TEST DETAILS <======================================")
    
    ## Metrics
    print(f"Train Size: {x_train.shape[0]} samples")
    print(f" Test Size: {x_test.shape[0]} samples")
    print("---------------------------------------------")
    print(f"Training Time: {round(train_time, 3)} seconds")
    print(f" Testing Time: {round(test_time, 3)} seconds")
    print("---------------------------------------------")
    print("Train Accuracy: ", train_acc)
    print(" Test Accuracy: ", test_acc)
    print("---------------------------------------------")
    
    if display_scores:
        for k, v in scores_dict.items():
            score_name = ' '.join(map(lambda x: x.title(), k.split('_')))
            print(f'Train {score_name}: ', v[0])
            print(f' Test {score_name}: ', v[1])
            print()
        print("---------------------------------------------") 
    print(" Area Under ROC (test): ", roc_auc)
    print("---------------------------------------------")
    print(f"Model Memory Size: {model_mem} kB")
    print("\n=============================> CLASSIFICATION REPORT <===================================")
    
    ## Classification Report
    clf_rep = classification_report(y_test, test_predictions, output_dict=True)
    
    print(classification_report(y_test, test_predictions,
                                target_names=confusion_labels))
    
    
    if verbose:
        print("\n================================> CONFUSION MATRIX <=====================================")
    
        ## Confusion Matrix HeatMap
        display(confusion_plot(confusion_matrix(y_test, test_predictions),
                               labels=confusion_labels))
        print("\n=======================================> PLOTS <=========================================")


        ## Variable importance plot
        fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 10))
        roc_axes = axes[0, 0]
        pr_axes = axes[0, 1]
        importances = None

        if importance_plot:
            if not feature_labels:
                raise RuntimeError("'feature_labels' argument not passed "
                                   "when 'importance_plot' is True")

            try:
                importances = pd.Series(clf.feature_importances_,
                                        index=feature_labels) \
                                .sort_values(ascending=False)
            except AttributeError:
                try:
                    importances = pd.Series(clf.coef_.ravel(),
                                            index=feature_labels) \
                                    .sort_values(ascending=False)
                except AttributeError:
                    pass

            if importances is not None:
                # Modifying grid
                grid_spec = axes[0, 0].get_gridspec()
                for ax in axes[:, 0]:
                    ax.remove()   # remove first column axes
                large_axs = fig.add_subplot(grid_spec[0:, 0])

                # Plot importance curve
                feature_importance_plot(importances=importances.values,
                                        feature_labels=importances.index,
                                        ax=large_axs)
                large_axs.axvline(x=0)

                # Axis for ROC and PR curve
                roc_axes = axes[0, 1]
                pr_axes = axes[1, 1]
            else:
                # remove second row axes
                for ax in axes[1, :]:
                    ax.remove()
        else:
            # remove second row axes
            for ax in axes[1, :]:
                ax.remove()


        ## ROC and Precision-Recall curves
        clf_name = clf.__class__.__name__
        roc_plot(y_test, y_probs, clf_name, ax=roc_axes)
        precision_recall_plot(y_test, y_probs, clf_name, ax=pr_axes)

        fig.subplots_adjust(wspace=5)
        fig.tight_layout()
        display(fig)
    
    ## Dump to report_dict
    dump = dict(clf=clf, accuracy=[train_acc, test_acc], **scores_dict,
                train_time=train_time, train_predictions=train_predictions,
                test_time=test_time, test_predictions=test_predictions,
                test_probs=y_probs, report=clf_rep, roc_auc=roc_auc,
                model_memory=model_mem)
    
    return clf, dump

**Main code-model tuning using optuna**

In [ ]:
X = newvar.drop(['Anemia','Weight','Anemia_treatment'],axis=1)
y = newvar[['Anemia']]

In [ ]:

import xgboost as xgb
from optuna import create_study, logging
from optuna.pruners import MedianPruner
from optuna.integration import XGBoostPruningCallback
from xgboost import XGBClassifier
from sklearn.utils import class_weight
from sklearn import metrics

from sklearn.model_selection import train_test_split
#deopping hemoglobin

data_splits = train_test_split(X, y, test_size=0.3, random_state=123,
                               shuffle=True, stratify=y)
x_train, x_test, y_train, y_test = data_splits
def objective(trial, X, y, group, score, params=dict()):
    dtrain = xgb.DMatrix(X, label=y)
    class_weight = (y.shape[0] - np.sum(y)) / np.sum(y)
    
    ## Initial Learning Parameters
    params['learning_rate'] = 0.1
    params['num_boost_round'] = 1000

    if group == '1':
        params['max_depth'] = trial.suggest_int('max_depth', 2, 15)
        params['min_child_weight'] = trial.suggest_loguniform('min_child_weight',
                                                              1e-10, 1e10)
        params['n_estimators']=trial.suggest_int('n_estimators',500,1000)
        params['scale_pos_weight']=trial.suggest_float('scale_pos_weight',3.5,5)
    
    if group == '2':
        params['min_samples_split']=trial.suggest_int('min_samples_split',5,10)
        params['subsample'] = trial.suggest_uniform('subsample', 0, 1)
        params['colsample_bytree'] = trial.suggest_uniform('colsample_bytree', 0, 1)
    
    if group == '3':
        params['learning_rate'] = trial.suggest_uniform('learning_rate', 0, 0.1)
        params['num_boost_round'] = trial.suggest_int('num_boost_round', 100, 1000)

    pruning_callback = XGBoostPruningCallback(trial, "test-" + score.__name__)
    cv_scores = xgb.cv(params, dtrain, nfold=5,
                       stratified=True,
                       feval=score,
                       early_stopping_rounds=10,
                       callbacks=[pruning_callback],
                       seed=0)

    return cv_scores['test-' + score.__name__ + '-mean'].values[-1]
def execute_optimization(study_name, group, score, trials,
                         params=dict(), direction='maximize'):
    logging.set_verbosity(logging.ERROR)
    
    ## We use pruner to skip trials that are NOT fruitful
    pruner = MedianPruner(n_warmup_steps=5)
    
    study = create_study(direction=direction,
                         study_name=study_name,
                         storage='sqlite:///optuna.db',
                         load_if_exists=True,
                         pruner=pruner)

    study.optimize(lambda trial: objective(trial, x_train, y_train,
                                           group, score, params),
                   n_trials=trials,
                   n_jobs=-1)
    
    
    print("STUDY NAME: ", study_name)
    print('------------------------------------------------')
    print("EVALUATION METRIC: ", score.__name__)
    print('------------------------------------------------')
    print("BEST CV SCORE", study.best_value)
    print('------------------------------------------------')
    print(f"OPTIMAL GROUP - {group} PARAMS: ", study.best_params)
    print('------------------------------------------------')
    print("BEST TRIAL", study.best_trial)
    print('------------------------------------------------')
    
    
    return study.best_params


In [ ]:
score_func = metrics.f1_score
def score_function(y_pred, dtrain):
    y_pred = (y_pred > 0.5).astype(int)
    y_true = (dtrain.get_label() > 0.5).astype(int)
    return score_func.__name__, score_func(y_true, y_pred)

score_function.__name__ = score_func.__name__

stepwise_optimization(trials=10)

In [ ]:
def stepwise_optimization(trials=10):
    final_params = dict()
    for g in ['1', '2', '3']:
        print(f"=========================== Optimizing Group - {g} ============================")
        update_params = execute_optimization('xgboost', g, score_function, trials,
                                             params=final_params, direction='maximize')
        final_params.update(update_params)
        print(f"PARAMS after optimizing GROUP - {g}: ", final_params)
        print()
        print()

    print("=========================== FINAL OPTIMAL PARAMETERS ============================")
    print(final_params)
    
    return final_params

In [ ]:
params1 = stepwise_optimization()

In [ ]:
params1

In [ ]:
params1={'learning_rate': 0.02578297296713958,
 'num_boost_round': 585,
 'max_depth': 7,
 'min_child_weight': 18.93704847111416,
 'n_estimators': 754,
         'min_samples_split': 9,
 'scale_pos_weight': 3.425510435559797,
 'subsample': 0.505407237617943,
 'colsample_bytree': 0.5538227649946694}

In [ ]:
primary_eval_metric = metrics.f1_score

In [ ]:
n=list(x_train.columns)

In [ ]:
X

In [ ]:
xgb_clf_tuned_1 = XGBClassifier(**params1, 
                                random_state=45, n_jobs=-1)


In [ ]:
xgb_clf_tuned_1.fit(x_train, y_train);


 

In [ ]:
xgb_clf_tuned_1, xgb_report_tuned_1 = report(xgb_clf_tuned_1, x_train, y_train,
                                             x_test, y_test,
                                             display_scores=[primary_eval_metric],
                                             importance_plot=True,
                                             feature_labels=n
                                             )

In [ ]:
    ## Classification Report
    ##clf_rep = classification_report(y_test, test_predictions, output_dict=True)
    c=xgb_clf_tuned_1.fit(x_train, y_train);
    test_predictions = c.predict(x_test)
    print(classification_report(y_test, test_predictions
                                ))
    #prediction and Classification Report

    

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_train, c.predict(x_train)))

In [ ]:
params2 = stepwise_optimization(trials=50)

In [ ]:
params2

In [ ]:
xgb_clf_tuned_2 = XGBClassifier(**params2,
                                 random_state=45, n_jobs=-1)
xgb_clf_tuned_2.fit(x_train, y_train);

xgb_clf_tuned_2, xgb_report_tuned_2 = report(xgb_clf_tuned_2, x_train, y_train,
                                             x_test, y_test,
                                             display_scores=[primary_eval_metric],
                                             importance_plot=True,
                                             feature_labels=n
                                             )

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : estimator instance
        An estimator instance implementing `fit` and `predict` methods which
        will be cloned for each validation.

    title : str
        Title for the chart.

    X : array-like of shape (n_samples, n_features)
        Training vector, where ``n_samples`` is the number of samples and
        ``n_features`` is the number of features.

    y : array-like of shape (n_samples) or (n_samples, n_features)
        Target relative to ``X`` for classification or regression;
        None for unsupervised learning.

    axes : array-like of shape (3,), default=None
        Axes to use for plotting the curves.

    ylim : tuple of shape (2,), default=None
        Defines minimum and maximum y-values plotted, e.g. (ymin, ymax).

    cv : int, cross-validation generator or an iterable, default=None
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:

          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, default=None
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like of shape (n_ticks,)
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the ``dtype`` is float, it is regarded
        as a fraction of the maximum size of the training set (that is
        determined by the selected validation method), i.e. it has to be within
        (0, 1]. Otherwise it is interpreted as absolute sizes of the training
        sets. Note that for classification the number of samples usually have
        to be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                         fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt


fig, axes = plt.subplots(3, 2, figsize=(10, 15))

X, y = load_digits(return_X_y=True)

title = "Learning Curves (XGBclassifiermodel1)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

estimator = XGBClassifier(**params1, 
                                random_state=45, n_jobs=-1)
plot_learning_curve(estimator, title, X, y, axes=axes[:, 0], ylim=(0.7, 1.01),
                    cv=cv, n_jobs=4)

title = r"Learning Curves (XGBclassifier2, $\gamma=0.001$)"
# SVC is more expensive so we do a lower number of CV iterations:
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
estimator =XGBClassifier(**params2,
                                 random_state=45, n_jobs=-1)
plot_learning_curve(estimator, title, X, y, axes=axes[:, 1], ylim=(0.7, 1.01),
                    cv=cv, n_jobs=4)

plt.show()

**SHAP**

In [ ]:
import shap
import xgboost
from sklearn.model_selection import train_test_split
import matplotlib.pylab as pl

In [ ]:
#deopping hemoglobin
X = df_m1.drop(['Anemia','Hemoglobin','Anemia_treatment'],axis=1)
y = df_m1[['Anemia']]

In [ ]:
X

In [ ]:
shap_values = shap.TreeExplainer(xgb_clf_tuned_1).shap_values(x_train)

In [ ]:
shap.summary_plot(shap_values, x_train)

In [ ]:
shap.dependence_plot("GlycoHemoglobin", shap_values, x_train,interaction_index=None)

In [ ]:
shap.dependence_plot("Gender", shap_values, x_train,interaction_index=None)

In [ ]:
shap.dependence_plot("Age", shap_values, x_train,interaction_index=None)

In [ ]:
shap.dependence_plot("BMI", shap_values, x_train,interaction_index=None)

In [ ]:
shap.dependence_plot("DRD360", shap_values, x_train,interaction_index=None)

In [ ]:
shap.dependence_plot("DR1DRSTZ", shap_values, x_train,interaction_index=None)

**LIGHTGBM**

In [ ]:
import scipy
import scipy.stats
from scipy import stats

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import gc
seed = 195
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
evals = {}
evals['lgb'] = {}
lparams = {}
seeds = [0, 1]
nfold = [5, 5]
stratified=[False, False]
shuffle=[True, True]
n_estimators = 30000
early_stopping_rounds = 300
verbose_eval = 0
learning_rate = 0.01
reg_alpha = [0.4, 1]
reg_lambda = [0.7, 1]
subsample = [0.45, 1]
colsample_bytree = [0.3, 0.225]
max_depth = -1
verbose = -1
n_jobs = 4
lparams[0] = dict(boosting_type='gbdt',
               objective='binary',
               metric='auc',
               learning_rate= learning_rate,
               num_leaves= 200,
               max_bin=500,
               min_child_weight= 0.035,
               subsample= subsample[0],
               colsample_bytree= colsample_bytree[0],
               min_data_in_leaf= 150,
               max_depth= max_depth,
               bagging_seed= seeds[0],
               reg_alpha= reg_alpha[0],
               reg_lambda= reg_lambda[0],
               verbose= verbose,
               seed= seeds[0],
               n_jobs= n_jobs,)
lparams[1] = dict(boosting_type='gbdt',
               objective='binary',
               metric='auc',
               learning_rate= learning_rate,
               n_estimators= n_estimators,
               subsample= subsample[1],
               colsample_bytree= colsample_bytree[1],
               max_depth= max_depth,
               bagging_seed= seeds[1],
               reg_alpha= reg_alpha[1],
               reg_lambda= reg_lambda[1],
               verbose= verbose,
               seed= seeds[1],
               n_jobs= n_jobs,)
test_preds = np.zeros(len(X_test))    
dtrain = lgb.Dataset(X_train, y_train)
dtest = X_test.copy()
testlen = X_test.shape[0]

gc.collect()
for i, seed in enumerate(seeds):
    print(f'Training Model with SEED : {seed}')
    evals['lgb'][i] = lgb.cv(lparams[i],
                             dtrain,
                             nfold=nfold[i], 
                             stratified=stratified[i],
                             shuffle=shuffle[i],
                             num_boost_round=n_estimators,
                             early_stopping_rounds=early_stopping_rounds,
                             verbose_eval=verbose_eval,
                             return_cvbooster=True,
                             seed = seed,
                             show_stdv=True)
    
                  
#     filename = 'lgb_'+ i+'_.sav'
#     pickle.dump(evals['lgb'][i], open(filename, 'wb'))
    print(f'SEED {i} Average fold  AUC {np.round(max(evals["lgb"][i]["auc-mean"]),5)}')
    
    
    test_preds += stats.rankdata(np.mean(evals['lgb'][i]['cvbooster'].predict(dtest, num_iteration=evals['lgb'][i]['cvbooster'].best_iteration), axis=0)) / (testlen * len(seeds))
    

In [ ]:
Model= lgb.train(params=lparams[1],train_set=dtrain)

#prediction and Classification Report
from sklearn.metrics import classification_report

pred = Model.predict(X_test)
pred_labels = np.rint(pred)
   
mean_accuracy=sklearn.metrics.accuracy_score(y_test, pred_labels),

print( (classification_report(y_test, pred_labels)))

In [ ]:
from sklearn.metrics import roc_auc_score
auc_score1 = roc_auc_score(y_test, pred_labels)
auc_score1

**XGBClassifier with K-fold cross validation**

In [ ]:
model1=xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=500,
 max_depth=6,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 scale_pos_weight=4.747735,
 seed=27,eval_metric="error")
train_model = model1.fit(X_train, y_train,eval_metric="error",verbose=True)

In [ ]:
# stratified k-fold cross validation evaluation of xgboost model
from numpy import loadtxt
import xgboost
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

# CV model
kfold = StratifiedKFold(n_splits=5)
results = cross_val_score(model1, X, y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
# import XGBoost
import xgboost as xgb


# define data_dmatrix
data_dmatrix = xgb.DMatrix(data=X,label=y)
from xgboost import cv

params = {'learning_rate':0.1,
 'n_estimators':500,
 'max_depth':6,
 'min_child_weight':1,
 'gamma':0,
 'subsample':0.8,
 'colsample_bytree':0.8,
 'objective': 'binary:logistic',
 'scale_pos_weight':4.747735,
 'seed':27}

xgb_cv = cv(dtrain=data_dmatrix, params=params, nfold=5,num_boost_round=5,
                     metrics="error", as_pandas=True, seed=123)

In [ ]:
print(((1-xgb_cv["test-error-mean"]).iloc[-1]))


In [ ]:
xgb_cv.head()

**XGBClassifier**

In [ ]:
model1=xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=500,
 max_depth=6,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 scale_pos_weight=4.747735,
 seed=27,eval_metric="error")
train_model = model1.fit(X_train, y_train,eval_metric="error",verbose=True)

In [ ]:
#prediction and Classification Report
from sklearn.metrics import classification_report

pred = train_model.predict(X_test)

print( (classification_report(y_test, pred)))

In [ ]:
print(classification_report(y_train, train_model.predict(X_train)))

In [ ]:
from sklearn.metrics import accuracy_score
print('Accuracy: {0:.2f}'.format(accuracy_score(y_test, pred)))
#print('Precision: {0:.2f}'.format(precision_score(y_test, pred)))
#print('Recall: {0:.2f}'.format(recall_score(y_test, pred)))
#print('AUC ROC: {0:.2f}'.format(roc_auc_score(y_test, pred)))

In [ ]:
xgb.plot_importance(train_model)
plt.rcParams['figure.figsize'] = [15,9]
plt.show()

**RANDOM FOREST CLASSIFIER**

In [ ]:
#RAndom forest classifier
# import Random Forest classifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc


n_estimators = [1, 2, 4, 8, 16, 32, 64, 100, 200,500,1000]
train_results = []
test_results = []
for estimator in n_estimators:
   rf = RandomForestClassifier(n_estimators=estimator, n_jobs=-1)
   rf.fit(X_train, y_train)
   train_pred = rf.predict(X_train)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   train_results.append(roc_auc)
   y_pred = rf.predict(X_test)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   test_results.append(roc_auc)
    
from matplotlib.legend_handler import HandlerLine2D
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

line1, = plt.plot(n_estimators, train_results, "b", label="Train AUC")
line2, = plt.plot(n_estimators, test_results, "r", label="Test AUC")
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel("AUC score")
plt.xlabel("n_estimators")
plt.show()

In [ ]:
max_depths = np.linspace(1, 32, 32, endpoint=True)
train_results = []
test_results = []
for max_depth in max_depths:
   rf = RandomForestClassifier(max_depth=max_depth, n_jobs=-1)
   rf.fit(X_train, y_train)
   train_pred = rf.predict(X_train)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   train_results.append(roc_auc)
   y_pred = rf.predict(X_test)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   test_results.append(roc_auc)
from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(max_depths, train_results, "b", label="Train AUC")
line2, = plt.plot(max_depths, test_results, "r", label="Test AUC" )
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel("AUC score")
plt.xlabel("Tree depth")
plt.show()

In [ ]:
min_samples_leafs = np.linspace(0.1,  0.5,50 ,endpoint=True)
train_results = []
test_results = []
for min_samples_leaf in min_samples_leafs:
   rf = RandomForestClassifier(min_samples_leaf=min_samples_leaf)
   rf.fit(X_train, y_train)
   train_pred = rf.predict(X_train)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   train_results.append(roc_auc)
   y_pred = rf.predict(X_test)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   test_results.append(roc_auc)
from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(min_samples_leafs, train_results,"b",label="Train AUC")
line2, = plt.plot(min_samples_leafs, test_results, "r", label="Test AUC")
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel("AUC score")
plt.xlabel("min samples leaf")
plt.show()

In [ ]:
#RAndom forest classifier
# import Random Forest classifier

from sklearn.ensemble import RandomForestClassifier
#Split the dataset into train and Test
from sklearn.model_selection import train_test_split

seed = 195
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)


# instantiate the classifier 

rfc = RandomForestClassifier(class_weight="balanced", n_estimators=500, min_samples_leaf=50, max_depth=7)



# fit the model

rfc.fit(X_train, y_train)



# Predict the Test set results

y_pred = rfc.predict(X_test)



# Check accuracy score 

from sklearn.metrics import accuracy_score

print('Model accuracy score with  decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))


In [ ]:
#prediction and Classification Report
from sklearn.metrics import classification_report

pred = rfc.predict(X_test)

print( (classification_report(y_test, y_pred)))

# view the feature scores

feature_scores = pd.Series(rfc.feature_importances_, index=X_train.columns).sort_values(ascending=False)

feature_scores

     n_estimators=500 minsample leaf 50
     precision    recall  f1-score   support

         0.0       0.90      0.81      0.85      7174
         1.0       0.39      0.56      0.46      1527

    accuracy                           0.77      8701
   macro avg       0.64      0.69      0.66      8701
weighted avg       0.81      0.77      0.78      8701


  max-depth 6 subsample 0.8
      precision    recall  f1-score   support

         0.0       0.89      0.84      0.86      7174
         1.0       0.40      0.50      0.44      1527

    accuracy                           0.78      8701
   macro avg       0.64      0.67      0.65      8701
weighted avg       0.80      0.78      0.79      8701


Accuracy for model : 78.08



subsample 0.5 max depth 10


precision    recall  f1-score   support

         0.0       0.86      0.91      0.89      7174
         1.0       0.43      0.32      0.37      1527

    accuracy                           0.81      8701
   macro avg       0.65      0.61      0.63      8701
weighted avg       0.79      0.81      0.79      8701


max-depth 6 subsample 0.8 
  precision    recall  f1-score   support

         0.0       0.87      0.89      0.88      7174
         1.0       0.43      0.39      0.41      1527

    accuracy                           0.80      8701
   macro avg       0.65      0.64      0.65      8701
weighted avg       0.80      0.80      0.80      8701
Accuracy for model : 80.20